# MNIST
Датасет от MNIST(Modified National Institute of Standards and Technology) - это несколько десятков тысяч картинок с рукописными цифрами 28х28. Для начала его нужно скачать и переместить в папку с данным ноутбуком папку из архива.

[Ссылка на скачивание архива](https://www.kaggle.com/scolianni/mnistasjpg/downloads/trainingSet.tar.gz/1)

Далее сделаем ```.rec``` файл из имеющихся данных, чтобы передать их нейросети. Воспользуемся файлом ```im2rec.py```. Пример директории, где он должен лежать:

anaconda3/lib/python3.6/site-packages/mxnet/tools/im2rec.py

Нам потребуются файлы ```.lst``` для генерации ```.rec```. Чтобы их создать нужно:

```python path_to_im2rec.py --list --recursive --train-ratio=0.9 --test-ratio=0.1 prefix path_to_images```

Пояснение:
* ```--list``` означает, что нам нужно сгенерировать файл ```.lst```
* параметр ```--recursive``` включает рекурсивный проход по вложенным папкам и даёт файлам уникальные названия, в зависимости от папки, где они лежат
* ```--train-ratio``` и ```--test-ratio``` отвечают за то, какой процент от имеющихся файлов будет отведён для обучения нейросети, а какой для валидации.
* prefix - какой префикс будут иметь сгенерированные файлы
* path_to_images - путь до папки с изображениями

Пример для нашего случая:

```python anaconda3/lib/python3.6/site-packages/mxnet/tools/im2rec.py --list --recursive --train-ratio=0.9 --test-ratio=0.1 mnist trainingSet/```

Затем генерация .rec файла. Синтаксис:

```python path_to_im2rec.py --color={-1,0,1} prefix path_to_images```

Пояснения:
* ```color``` - режим цвета для изображений. 1 - загружается полностью цвет изображения, прозрачность игнорируется, 0 - оттенки серого, -1 - Загружает изображение, включая альфа-канал.
* prefix - префикс ```.lst``` файлов
* path_to_images - путь до папки с изображениями, откуда генерировались ```.lst``` файлы.

Наш случай:

```python ~/anaconda3/lib/python3.6/site-packages/mxnet/tools/im2rec.py --color=0  mnist trainingSet/```

В итоге файлы ```.rec``` должны в сумме иметь такой же объём, какой и у папки с изображениями.

Можно приступать к созданию слоёв.

In [10]:
import mxnet as mx
import logging

logging.basicConfig(level=logging.INFO)

In [11]:
data = mx.sym.Variable('data')
data = mx.sym.Flatten(data=data)
fc1  = mx.sym.FullyConnected(data=data, name='fc1', num_hidden=128)
act1 = mx.sym.Activation(data=fc1, name='relu1', act_type="relu")
fc2  = mx.sym.FullyConnected(data=act1, name='fc2', num_hidden=10)
mlp  = mx.sym.SoftmaxOutput(data=fc2, name='softmax')
mod = mx.mod.Module(mlp)

Воспользуемся специальным итератором по rec файлам. Один будет для тренировки нейросети, другой для валидации.

In [12]:
train_iter = mx.io.ImageRecordIter(path_imglist='./mnist_train.lst', 
                                   path_imgrec='./mnist_train.rec',
                                   data_shape=(1, 28, 28), #кортеж формата (число каналов, высота, ширина)
                                   batch_size=100)          #размер пакета

valid_iter = mx.io.ImageRecordIter(path_imglist='./mnist_test.lst', 
                                   path_imgrec='./mnist_test.rec',
                                   data_shape=(1, 28, 28),
                                   batch_size=100)

Попробуем запустить с такими параметрами.

In [13]:
mod.fit(train_data=train_iter,
         eval_data=valid_iter,
        initializer=mx.initializer.Xavier(magnitude=2.),
        optimizer='sgd',
        optimizer_params={'learning_rate': 0.1},
        num_epoch=10)

INFO:root:Epoch[0] Train-accuracy=0.110820
INFO:root:Epoch[0] Time cost=0.730
INFO:root:Epoch[0] Validation-accuracy=0.105476
INFO:root:Epoch[1] Train-accuracy=0.112354
INFO:root:Epoch[1] Time cost=0.917
INFO:root:Epoch[1] Validation-accuracy=0.105476
INFO:root:Epoch[2] Train-accuracy=0.112196
INFO:root:Epoch[2] Time cost=0.758
INFO:root:Epoch[2] Validation-accuracy=0.105476
INFO:root:Epoch[3] Train-accuracy=0.112196
INFO:root:Epoch[3] Time cost=1.030
INFO:root:Epoch[3] Validation-accuracy=0.105476
INFO:root:Epoch[4] Train-accuracy=0.112196
INFO:root:Epoch[4] Time cost=0.941
INFO:root:Epoch[4] Validation-accuracy=0.105476
INFO:root:Epoch[5] Train-accuracy=0.112196
INFO:root:Epoch[5] Time cost=1.092
INFO:root:Epoch[5] Validation-accuracy=0.105476
INFO:root:Epoch[6] Train-accuracy=0.112196
INFO:root:Epoch[6] Time cost=1.238
INFO:root:Epoch[6] Validation-accuracy=0.105476
INFO:root:Epoch[7] Train-accuracy=0.112196
INFO:root:Epoch[7] Time cost=1.056
INFO:root:Epoch[7] Validation-accuracy=0

За 10 эпох сеть "ничему" не научилась. Изменим learning_rate на меньшее значение.

In [14]:
train_iter.reset()
valid_iter.reset() #но сначала перезагрузим итераторы
mod.init_optimizer(force_init=True, #чтобы присвоить новые параметры
                   optimizer_params=(('learning_rate', 0.01),), 
                   optimizer='sgd')
mod.init_params(force_init=True, 
                initializer=mx.initializer.Xavier(magnitude=2.))

In [15]:
mod.fit(train_data=train_iter,
         eval_data=valid_iter,
        num_epoch=10)

/home/gleb/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.413598
INFO:root:Epoch[0] Time cost=1.259
INFO:root:Epoch[0] Validation-accuracy=0.556667
INFO:root:Epoch[1] Train-accuracy=0.579630
INFO:root:Epoch[1] Time cost=1.039
INFO:root:Epoch[1] Validation-accuracy=0.630714
INFO:root:Epoch[2] Train-accuracy=0.550926
INFO:root:Epoch[2] Time cost=1.089
INFO:root:Epoch[2] Validation-accuracy=0.614048
INFO:root:Epoch[3] Train-accuracy=0.632063
INFO:root:Epoch[3] Time cost=0.828
INFO:root:Epoch[3] Validation-accuracy=0.647381
INFO:root:Epoch[4] Train-accuracy=0.648598
INFO:root:Epoch[4] Time cost=1.235
INFO:root:Epoch[4] Validation-accuracy=0.616429
INFO:root:Epoch[5] Train-accuracy=0.604286
INFO:root:Epoch[5] Time cost=0.998
INFO:root:Epoch[5] Validation-accuracy=0.620238
INFO:root:Epoch[

Уменьшим ещё и дадим больше времени на обучение.

In [18]:
train_iter.reset()
valid_iter.reset() 
mod.init_optimizer(force_init=True,
                   optimizer_params=(('learning_rate', 0.001),), 
                   optimizer='sgd')
mod.init_params(force_init=True, 
                initializer=mx.initializer.Xavier(magnitude=2.))

In [19]:
mod.fit(train_data=train_iter,
         eval_data=valid_iter,
        num_epoch=25)

/home/gleb/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.730556
INFO:root:Epoch[0] Time cost=0.814
INFO:root:Epoch[0] Validation-accuracy=0.804762
INFO:root:Epoch[1] Train-accuracy=0.833360
INFO:root:Epoch[1] Time cost=1.123
INFO:root:Epoch[1] Validation-accuracy=0.842857
INFO:root:Epoch[2] Train-accuracy=0.868730
INFO:root:Epoch[2] Time cost=0.943
INFO:root:Epoch[2] Validation-accuracy=0.873095
INFO:root:Epoch[3] Train-accuracy=0.890000
INFO:root:Epoch[3] Time cost=2.861
INFO:root:Epoch[3] Validation-accuracy=0.882143
INFO:root:Epoch[4] Train-accuracy=0.903862
INFO:root:Epoch[4] Time cost=2.207
INFO:root:Epoch[4] Validation-accuracy=0.889524
INFO:root:Epoch[5] Train-accuracy=0.914180
INFO:root:Epoch[5] Time cost=2.001
INFO:root:Epoch[5] Validation-accuracy=0.893333
INFO:root:Epoch[

# В чём тут дело.

learning rate - это то, как нейросеть быстро отвергает то, чему она научилась, в угоду новому.

Если ребёнок увидит 10 кошек с оранжевой шерстью, то будет думать, что это их отличительная черта, и в попытке узнать кота, начнёт искать оранжевую шерсть. Теперь он увидит чёрную кошку и родители скажут, что это кошка (учение под присмотром - supervised learning). С большим "learning rate", он быстро осознает, что "рыжая шерсть" не самый важный "параметр", свойство кошек. С маленьим learning rate он будет думать, что чёрная кошка просто "выброс", и кошки продолжают быть оранжевыми.

Высокое значение скорости обучения может быть подходящим для описанного случчая. Но если оно будет слишком высоким, то ребёнок может начать думать, что все кошки чёрные, даже увидев до этого много рыжих.

## По-другому
Нейронная сеть тренируется применяя к весам метод градиентного спуска. Это значит, что каждую итерацию мы будем использовать обратное рапсротранение, чтобы посчитать производную функции потерь по каждому весу и вычесть из текущего веса. Если попробовать это сделать, веса будут меняться очень сильно на каждом шаге, что сделает их неверными и потери возрастут. На практике, люди умножают каждую производную на малое значение, learning rate или скорость обучения, до того как вычитать.